#### Single Document NER

We expect as input a PDF file which will be converted to a raw text format using *pdfminer*, the text extraction tool selected for this task given that it extracts the most information when compared to others such as *Py2PDF* (used in the word2vec part of the project).

In [86]:
from pdfminer.layout import LTTextContainer
from pdfminer.high_level import extract_text, extract_pages

def extract_text_from_pdf(pdf_file):
    page_counter = 0
    text_as_list = []
    text_as_str = ''

    for page_layout in extract_pages(pdf_file):
        page_counter += 1
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                extracted_text = element.get_text()
                text_as_list.append(extracted_text)
                text_as_str += '' + extracted_text
    print('Text from PDF File ({} pages) extracted successfully.'.format(page_counter)) 
    return text_as_str, text_as_list

def try_finding_keywords(document_text_str):
    potential_keywords_index = document_text_str.find('Keywords')
    final_keywords = []

    if potential_keywords_index != -1:
        found_keywords = []
        for i in range(potential_keywords_index, len(document_text_str)):
            if document_text_str[i] != '\n':
                found_keywords.append(document_text_str[i])
            else:
                break     
                
        keyword = ''
        for i in range(len(found_keywords)):
            if found_keywords[i].isalpha():
                keyword += found_keywords[i]
            elif (keyword != ''):
                final_keywords.append(keyword)
                keyword = ''    
    
    return final_keywords

Loop to treat all publications in our testing set.

In [94]:
import os

pdfs_directory = 'data/implementome_publications/test_miner/'
#### Load the paths of the input PDF file and the output empty text notebook
for file in os.listdir(pdfs_directory):
    filename = os.fsdecode(file)
    # print('what are you', filename)
    if filename.endswith('.pdf'):
        print('For Publication: {}'.format(filename[:-4]))
        pdf_file = open(pdfs_directory + filename, 'rb')
        doc_as_str, doc_as_list = extract_text_from_pdf(pdf_file)
        print('Potential References Index: {}'.format(doc_as_str.find('References')))
        print('Potential References Index: {}'.format(doc_as_str.find('REFERENCES')))
        doc_keywords = try_finding_keywords(doc_as_str)
        print('Potential Extracted Keywords: {}'.format(doc_keywords[1:] if doc_keywords != [] else 'None Found'))
        print('')


# pdf_file = open('data/implementome_publications/test_miner/child_obesity_switzerland.pdf', 'rb')
# empty_text_file = open('data/implementome_publications/test_miner/test_text.txt', "w", encoding="utf-8")

# #### The extract_text() function is used to extract any text found in the pdf file
# #### Text in tables, headers, banners and other graphical representations is also extracted
# publications_text = extract_text(pdf_file)
# print('Text from PDF File extracted successfully.')
# # empty_text_file.write(publications_text)

For Publication: ai_and_surgical_decision_making
Text from PDF File (11 pages) extracted successfully.
Potential References Index: -1
Potential References Index: 50476
Potential Extracted Keywords: None Found

For Publication: brachytherapy_lmic
Text from PDF File (10 pages) extracted successfully.
Potential References Index: -1
Potential References Index: 38957
Potential Extracted Keywords: None Found

For Publication: child_obesity_switzerland
Text from PDF File (11 pages) extracted successfully.
Potential References Index: 36152
Potential References Index: -1
Potential Extracted Keywords: ['Overweight', 'Obesity', 'Migration', 'Children']

For Publication: haemophilia_senegal
Text from PDF File (7 pages) extracted successfully.
Potential References Index: -1
Potential References Index: -1
Potential Extracted Keywords: None Found

For Publication: humanitarian_crises_volunteerism
Text from PDF File (6 pages) extracted successfully.
Potential References Index: 23851
Potential Referenc

In [88]:
single_pdf = open('data/implementome_publications/test_miner/child_obesity_switzerland.pdf', 'rb')
doc_as_str, doc_as_list = extract_text_from_pdf(single_pdf)

Text from PDF File (11 pages) extracted successfully.


In [89]:
doc_as_str.find('References')

36152

In [90]:
doc_as_str[36152:]

'References\n1.\nGarrido-Miguel M, Cavero-Redondo I, Alvarez-Bueno C, Rodriguez-Artalejo F,\nMoreno LA, Ruiz JR, et al. Prevalence and trends of overweight and obesity\nin European children from 1999 to 2016: a systematic review and meta-\nanalysis. JAMA Pediatr. 2019;173:e192430.\n2.\n3.\n4.\n5.\n6.\n7.\n8.\n9.\nHerter-Aeberli I, Osuna E, Sarnovska Z, Zimmermann MB. Significant\ndecrease in childhood obesity and waist circumference over 15 years in\nSwitzerland: a repeated cross-sectional study. Nutrients. 2019;11:1922.\nEiholzer U, Fritz C, Katschnig C, Dinkelmann R, Stephan A. Contemporary\nheight, weight and body mass index references for children aged 0 to\nadulthood in Switzerland compared to the Prader reference, WHO and\nneighbouring countries. Ann Hum Biol. 2019;46:437–47.\nPrader A, Largo RH, Molinari L, Issler C. Physical growth of Swiss children\nfrom birth to 20 years of age. First Zurich longitudinal study of growth and\ndevelopment. Helv Paediatr Acta Suppl. 1989;52:1–12